In [ ]:
# Installing Libraries
!pip install tflearn

     |████████████████████████████████| 107 kB 26.6 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=465aef13558d04e5a7fc2f01ddc6421db3e24d6ee87c8a7db8e4ec26453fd180
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
#Imports
import nltk
nltk.download('punkt')
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#Loading the intents data
with open("intents.json") as file:
	data = json.load(file)

In [ ]:
#Initializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

In [ ]:
#Looping through our data
for intent in data['intents']:
	for pattern in intent['patterns']:
		pattern = pattern.lower()
    #Creating a list of words
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent['tag'])

	if intent['tag'] not in labels:
	  labels.append(intent['tag'])

In [ ]:
#Pre-Processing the given data
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1
	training.append(bag)
	output.append(output_row)
#Converting training data into NumPy arrays
training = np.array(training)
output = np.array(output)

In [ ]:
#Saving data to disk
with open("data.pickle","wb") as f:
	pickle.dump((words, labels, training, output),f)

In [ ]:
#Generating and Saving ML Model
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)
model.save("model.tflearn")

Training Step: 799  | total loss: 0.21169 | time: 0.031s
| Adam | epoch: 200 | loss: 0.21169 - acc: 0.9932 -- iter: 24/25
Training Step: 800  | total loss: 0.19156 | time: 0.037s
| Adam | epoch: 200 | loss: 0.19156 - acc: 0.9938 -- iter: 25/25
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
